In [ ]:
from utils import *
from yggdrasil.runner import run
from yggdrasil import tools

# Time Step Synchronization

In addition to allowing models to call other models as functions, yggdrasil also provides a method of synchronizing data across time dependent models, even if the models have different time steps. To see how this works in practice, the following shows how we could connect the plant model from above with the time-dependent root growth model shown below.

In [ ]:
from models import roots_v0
tools.display_source(roots_v0.run, number_lines=True)

First, the yaml for the new roots model will include a `timesync` parameter sepcifying the name of a dummy timesync "model" and an output entry for mass. 

In addition, the yaml shown below also includes the yaml entry for the dummy timesync "model" that specifies how data between models connecting to it should be aggregated (in this case by summing). This entry could be in the plant yaml or one of its own, but is included here for brevity.

In [ ]:
tools.display_source('yamls/roots.yml', number_lines=True)

The source code for the updated root model then contains two interface comms, one for the timesync and one for outputing masses. The name passed to the `YggTimesync` time step synchronization interface is the `timesync` param from the root model yaml and the name passed to the mass output interface is taken from the name of the output channel listed in the root model yaml. 

These interfaces are both used in time loop, with the `YggTimesync` interface being "called" using the same syntax as the `YggRpcClient` interface used in the previous server/client example.

In [ ]:
from models import roots_v1
tools.display_source_diff(roots_v0.run, roots_v1.run, number_lines=True)

To update the yaml for the plant model to include the time step synchronization, we only need to add a `timesync` parameter. By specifying the same name as the roots yaml, we indicate that data should be synchronized between those models. In this way, multiple synchronizations can be performed between different combinations of models.

In [ ]:
tools.display_source_diff('yamls/plant_v1.yml', 'yamls/plant_v2.yml', number_lines=True)

In the same way as the root model, time step synchronization is added to the plant model by adding a `YggTimesync` comm with calls inside the time loop.

In [ ]:
from models import plant_v2
tools.display_source_diff(plant_v1.run, plant_v2.run, number_lines=True)

This integration can then be run by providing the models for the updated plant model, light model, and root model to the yggdrasil `run` method.

In [ ]:
run(['yamls/plant_v2.yml', 'yamls/light.yml', 'yamls/roots.yml'], production_run=True)
display_last_timestep(with_light=True, with_masses=True)